In [1]:
# Cell 1: Imports & Load Cleaned Data
import pandas as pd
import os

# Ensure outputs folder exists
os.makedirs(r"C:\Users\Saket\OneDrive\Desktop\4th sem\python\EDA\EV_Market_Analysis\data/processed", exist_ok=True)

# Load cleaned datasets
ev_sales = pd.read_csv(r"C:\Users\Saket\OneDrive\Desktop\4th sem\python\EDA\EV_Market_Analysis\data/cleaned/ev_sales_cleaned.csv")
charging = pd.read_csv(r"C:\Users\Saket\OneDrive\Desktop\4th sem\python\EDA\EV_Market_Analysis\data/cleaned/charging_cleaned.csv")
gdp = pd.read_csv(r"C:\Users\Saket\OneDrive\Desktop\4th sem\python\EDA\EV_Market_Analysis\data/cleaned/gdp_cleaned.csv")

print("✅ Cleaned datasets loaded")


✅ Cleaned datasets loaded


In [2]:
# Cell 2: Inspect Shapes
print("EV Sales shape:", ev_sales.shape)
print("Charging shape:", charging.shape)
print("GDP shape:", gdp.shape)

print("\nEV Sales sample:\n", ev_sales.head(3))
print("\nCharging sample:\n", charging.head(3))
print("\nGDP sample:\n", gdp.head(3))


EV Sales shape: (492, 4)
Charging shape: (466, 40)
GDP shape: (58890, 4)

EV Sales sample:
      country iso_code  year  ev_share
0  Australia      AUS  2011    0.0061
1  Australia      AUS  2012    0.0290
2  Australia      AUS  2013    0.0330

Charging sample:
                                    the_geom  GIS Object ID  \
0  POINT (-76.593139599729 39.273002600332)            432   
1  POINT (-76.358197400357 39.522557499946)            415   
2  POINT (-77.015387000118 38.785151000271)            723   

  Alternative Fuel Type                  station_name       Street Address  \
0                  ELEC                  UNDER ARMOUR       1450 Beason St   
1                  ELEC                BMW OF BEL AIR        716 Belair Rd   
2                  ELEC  National Harbor Fleet Garage  150 Potomac Passage   

             Intersection Information       city state  zip_code  \
0                      CHEER BUILDING  Baltimore    MD     21230   
1  STATION SA01; back in service area  

In [3]:
# Cell 3: Feature Engineering for Charging Data

# Create total chargers column = Level1 + Level2 + Fast chargers
charging["total_chargers"] = (
    charging.get("level1_outlets", 0) +
    charging.get("level2_outlets", 0) +
    charging.get("fast_chargers", 0)
)

# Keep only relevant columns
charging_clean = charging[["state", "city", "latitude", "longitude", "total_chargers"]]

# 🚨 Since charging dataset is US-based (has city/state),
# we will aggregate chargers at COUNTRY level = "United States"
charging_clean["country"] = "United States"
charging_clean["year"] = pd.to_datetime(charging.get("Date Opened", "2020"), errors="coerce").dt.year.fillna(2020).astype(int)

# Aggregate chargers by country-year
charging_agg = charging_clean.groupby(["country", "year"], as_index=False)["total_chargers"].sum()

print("✅ Charging data aggregated by country-year")
charging_agg.head()


✅ Charging data aggregated by country-year


C:\Users\Saket\AppData\Local\Temp\ipykernel_23104\2652141547.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_clean["country"] = "United States"
C:\Users\Saket\AppData\Local\Temp\ipykernel_23104\2652141547.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  charging_clean["year"] = pd.to_datetime(charging.get("Date Opened", "2020"), errors="coerce").dt.year.fillna(2020).astype(int)
C:\Users\Saket\AppData\Local\Temp\ipykernel_23104\2652141547.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,country,year,total_chargers
0,United States,2011,26.0
1,United States,2012,54.0
2,United States,2013,19.0
3,United States,2014,26.0
4,United States,2015,105.0


In [4]:
# Cell 4: Merge EV Sales + GDP
ev_gdp = pd.merge(ev_sales, gdp, on=["country", "iso_code", "year"], how="left")

print("✅ EV + GDP merged")
ev_gdp.head()


✅ EV + GDP merged


,country,iso_code,year,ev_share,population
0,Australia,AUS,2011,0.0061,22479723.0
1,Australia,AUS,2012,0.0290,22852597.0
2,Australia,AUS,2013,0.0330,23236224.0
3,Australia,AUS,2014,0.1500,23595552.0
4,Australia,AUS,2015,0.1900,23947955.0


In [5]:
# Cell 5: Merge with Charging Infra
master = pd.merge(ev_gdp, charging_agg, on=["country", "year"], how="left")

# Fill missing chargers with 0
master["total_chargers"] = master["total_chargers"].fillna(0)

print("✅ Master dataset created")
master.head()


✅ Master dataset created


,country,iso_code,year,ev_share,population,total_chargers
0,Australia,AUS,2011,0.0061,22479723.0,0.0
1,Australia,AUS,2012,0.0290,22852597.0,0.0
2,Australia,AUS,2013,0.0330,23236224.0,0.0
3,Australia,AUS,2014,0.1500,23595552.0,0.0
4,Australia,AUS,2015,0.1900,23947955.0,0.0


In [6]:
# Cell 6: Create Derived Features

# Charger density = chargers per 100k people
master["charger_density_per_100k"] = (master["total_chargers"] / master["population"]) * 100000

# EV adoption rate already exists (ev_share %)
# But ensure it is percentage (0–100)
if master["ev_share"].max() <= 1:
    master["ev_share"] = master["ev_share"] * 100

# Population in millions (for scaling)
master["population_million"] = master["population"] / 1e6

print("✅ Derived features created")
master[["country", "year", "ev_share", "total_chargers", "charger_density_per_100k", "population_million"]].head()


✅ Derived features created


,country,year,ev_share,total_chargers,charger_density_per_100k,population_million
0,Australia,2011,0.0061,0.0,0.0,22.479723
1,Australia,2012,0.0290,0.0,0.0,22.852597
2,Australia,2013,0.0330,0.0,0.0,23.236224
3,Australia,2014,0.1500,0.0,0.0,23.595552
4,Australia,2015,0.1900,0.0,0.0,23.947955


In [7]:
# Cell 7: Save Master Dataset
master.to_csv(r"C:\Users\Saket\OneDrive\Desktop\4th sem\python\EDA\EV_Market_Analysis\data/processed/ev_master.csv", index=False)
print("✅ Master dataset saved at ../data/processed/ev_master.csv")


✅ Master dataset saved at ../data/processed/ev_master.csv


In [8]:
# Cell 8: Summary Stats
print(master.describe(include="all"))

print("\n📊 Master dataset ready for Day 4 (EDA visualizations)")


       country iso_code         year    ev_share    population  \
count      492      447   492.000000  492.000000  4.440000e+02   
unique      34       31          NaN         NaN           NaN   
top     Brazil      BEL          NaN         NaN           NaN   
freq        15       15          NaN         NaN           NaN   
mean       NaN      NaN  2017.239837    7.787549  4.084348e+08   
std        NaN      NaN     4.211733   14.904318  1.343228e+09   
min        NaN      NaN  2010.000000    0.000150  3.208020e+05   
25%        NaN      NaN  2014.000000    0.147500  1.014597e+07   
50%        NaN      NaN  2017.000000    1.200000  3.906013e+07   
75%        NaN      NaN  2021.000000    7.600000  1.267243e+08   
max        NaN      NaN  2024.000000   92.000000  8.091735e+09   

        total_chargers  charger_density_per_100k  population_million  
count       492.000000                444.000000          444.000000  
unique             NaN                       NaN                 

NameError: name 'master' is not defined